In [2]:
import os
from transformers import GPT2Model, GPT2Tokenizer, GPT2Config
PROJECT_ROOT = os.path.abspath(os.path.join(os.getcwd(), ".."))
# [핵심] PROJECT_ROOT 변수를 사용하면 경로가 훨씬 깔끔해집니다.
# (결과: /workspace/ship-ai/pretrained_models/gpt2)
SAVE_DIR = os.path.join(PROJECT_ROOT, "pretrained_models", "gpt2")

print(f"⬇️ Downloading GPT-2 model to: {SAVE_DIR}")

try:
    # 1. 모델/토크나이저 다운로드
    model = GPT2Model.from_pretrained('gpt2')
    tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
    config = GPT2Config.from_pretrained('gpt2')
    
    # 2. 저장
    model.save_pretrained(SAVE_DIR)
    tokenizer.save_pretrained(SAVE_DIR)
    config.save_pretrained(SAVE_DIR)
    
    print("✅ [SUCCESS] GPT-2 model downloaded and saved locally.")
    print(f"  > Path: {SAVE_DIR}")
    
except Exception as e:
    print(f"❌ [FAIL] Download Error: {e}")

⬇️ Downloading GPT-2 model to: /workspace/ship-ai/pretrained_models/gpt2
✅ [SUCCESS] GPT-2 model downloaded and saved locally.
  > Path: /workspace/ship-ai/pretrained_models/gpt2


In [3]:
import torch
import os
import sys

# 1. 경로 설정 (혹시 모르니 다시)
PROJECT_ROOT = os.path.abspath(os.path.join(os.getcwd(), ".."))
GPT2_PATH = os.path.join(PROJECT_ROOT, "pretrained_models", "gpt2")
TIME_LLM_ROOT = os.path.join(PROJECT_ROOT, "external", "time-llm")

if TIME_LLM_ROOT not in sys.path:
    sys.path.append(TIME_LLM_ROOT)

# 2. Time-LLM 임포트
try:
    import models.TimeLLM
    import importlib
    importlib.reload(models.TimeLLM) # 수정사항 반영을 위해 강제 리로드
    from models.TimeLLM import Model as TimeLLM
except ImportError:
    print("❌ Time-LLM import failed")

# 3. [최종] Configs 클래스 (빠짐없이 다 넣음!)
class Configs:
    def __init__(self):
        # [기본 설정]
        self.task_name = 'long_term_forecast'
        self.is_training = 1
        self.model_id = 'Stock_Prediction'
        self.model = 'TimeLLM'
        
        # [데이터 차원]
        self.seq_len = 120
        self.label_len = 60
        self.pred_len = 10
        self.enc_in = 12
        self.dec_in = 12
        self.c_out = 12
        
        # [LLM 설정]
        self.llm_model = 'GPT2' 
        self.llm_model_path = GPT2_PATH 
        self.llm_dim = 768       
        self.llm_layers = 12     
        
        self.patch_len = 16
        self.stride = 8
        self.d_model = 32
        self.d_ff = 128
        self.n_heads = 8
        self.dropout = 0.1
        
        # [!!! 에러 해결사: 누락된 설정들 추가 !!!]
        self.prompt_domain = 1   # <--- 아까 그 에러의 원인!
        self.content = "Predict the future stock price based on financial indicators."
        
        # (혹시 모를 다음 에러 방지용 안전장치들)
        self.embed = 'timeF'     # 시간 임베딩 방식
        self.freq = 'd'          # 데이터 빈도 (Daily)
        self.factor = 1          # Attention Factor
        self.moving_avg = 25     # Moving Average Kernel size
        self.e_layers = 2        # Encoder Layers
        self.d_layers = 1        # Decoder Layers
        self.top_k = 5           # Top-K optimization

# 4. 초기화 실행
if 'TimeLLM' in locals():
    print(f"⏳ Initializing Time-LLM (Final Attempt)...")
    try:
        configs = Configs()
        model = TimeLLM(configs)
        
        if torch.cuda.is_available():
            model = model.cuda()
            
        print("\n🎉 [SUCCESS] Real Time-LLM initialized! (드디어 성공!)")
        print(f"  > LLM Backbone: GPT-2 (from {configs.llm_model_path})")
        print(f"  > Trainable Params: {sum(p.numel() for p in model.parameters() if p.requires_grad):,}")
        
    except Exception as e:
        print(f"\n❌ [FAIL] Init Error: {e}")
        import traceback
        traceback.print_exc()

/opt/conda/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


⏳ Initializing Time-LLM (Final Attempt)...

🎉 [SUCCESS] Real Time-LLM initialized! (드디어 성공!)
  > LLM Backbone: GPT-2 (from /workspace/ship-ai/pretrained_models/gpt2)
  > Trainable Params: 52,674,650


In [5]:
import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np

# --- 1. 데이터셋 클래스 정의 (그릇 만들기) ---
class ShipDataset(Dataset):
    def __init__(self, X, y):
        # numpy 배열을 PyTorch 텐서로 변환 (GPU로 올리기 전 단계)
        self.X = torch.FloatTensor(X)
        self.y = torch.FloatTensor(y)
        
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

# --- 2. 데이터 분할 (Train / Val / Test) ---
# Phase 2에서 만든 X, y가 메모리에 있다고 가정합니다.
# (X shape: 1259, 120, 12)

if 'X_train_samsung' in locals(): # 변수명이 맞는지 확인하세요 (보통 X_samsung 등으로 저장됨)
    # 편의상 X, y로 이름 통일 (삼성중공업 데이터 사용)
    X_data = X_train_samsung  
    y_data = y_train_samsung
    
    total_samples = len(X_data)
    
    # 비율 설정 (7:1:2)
    train_size = int(total_samples * 0.7)
    val_size = int(total_samples * 0.1)
    test_size = total_samples - train_size - val_size
    
    # 시계열이므로 순서대로 자릅니다 (Shuffle=False)
    X_train = X_data[:train_size]
    y_train = y_data[:train_size]
    
    X_val = X_data[train_size : train_size+val_size]
    y_val = y_data[train_size : train_size+val_size]
    
    X_test = X_data[train_size+val_size:]
    y_test = y_data[train_size+val_size:]
    
    print(f"📊 Data Split Summary:")
    print(f"  > Total: {total_samples}")
    print(f"  > Train: {len(X_train)} samples (First 70%)")
    print(f"  > Val  : {len(X_val)} samples (Next 10%)")
    print(f"  > Test : {len(X_test)} samples (Last 20% - Future)")

    # --- 3. DataLoader 생성 (서빙 카트) ---
    BATCH_SIZE = 32
    
    train_dataset = ShipDataset(X_train, y_train)
    val_dataset = ShipDataset(X_val, y_val)
    test_dataset = ShipDataset(X_test, y_test)
    
    # 훈련 데이터는 섞어서(Shuffle) 학습 효과를 높입니다.
    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
    # 검증/테스트는 순서대로 평가합니다.
    val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)
    
    print(f"\n✅ [SUCCESS] DataLoaders ready!")
    print(f"  > Batch Size: {BATCH_SIZE}")
    
    # --- 4. (검증) 첫 번째 배치 꺼내보기 ---
    sample_batch = next(iter(train_loader))
    print(f"  > Sample Batch X shape: {sample_batch[0].shape}") # (32, 120, 12) 여야 함
    print(f"  > Sample Batch y shape: {sample_batch[1].shape}") # (32, 10, 12) 여야 함

else:
    print("❌ [ERROR] 'X_train_samsung' variable not found.")
    print("Phase 2-B 코드를 실행해서 윈도우 데이터를 먼저 생성해주세요!")

❌ [ERROR] 'X_train_samsung' variable not found.
Phase 2-B 코드를 실행해서 윈도우 데이터를 먼저 생성해주세요!
